In [1]:
from create_task import *

In [2]:
with open('dataset/amazon_laptop_sent.json', 'r') as f: laptop = json.load(f)
with open('dataset/amazon_laptop_vocab.json', 'r') as f: laptop_vocab = json.load(f)
with open('dataset/yelp_rest_sent.json', 'r') as f: rest = json.load(f)
with open('dataset/yelp_rest_vocab.json', 'r') as f: rest_vocab = json.load(f)

In [3]:
# 특정 빈도수 이하의 단어는 제거

min_freq = 500 
# SMLMT 논문에서는 50 사용
# 필요한 최소 빈도수: k+q = 80+10 = 90

laptop_vocab_freq = [tup for tup in laptop_vocab if tup[1]>=min_freq]
rest_vocab_freq = [tup for tup in rest_vocab if tup[1]>=min_freq]

print(len(laptop_vocab_freq))
print(len(rest_vocab_freq))

2593
3079


## Sample Vocab Randomly

#### Task 1개 생성 소요 시간
- n = 3, k = 80, q = 10, min_freq = 100 : 3분 45초
- n = 3, k = 80, q = 10, min_freq = 500 : 4분 6초 / 5 = 약 50초

#### To-Do
- domain = 'laptop', n = 2, min_freq = 500, num_task = 1000 // 8시간 37분 소요
- domain = 'laptop', n = 3, min_freq = 500, num_task = 1000 // 12시간 31분 소요
- domain = 'laptop', n = 4, min_freq = 500, num_task = 1000 // 매우 오래 걸림, 결과 저장 제대로 안 된듯
- domain = 'rest', n = 2, min_freq = 500, num_task = 1000 // 10시간 35분 소요
- domain = 'rest', n = 3, min_freq = 500, num_task = 1000 // 15시간 39분 소요
- domain = 'rest', n = 4, min_freq = 500, num_task = 1000 // 매우 오래 걸림 (laptop보다 더)

#### 종강 이후
- 이제부터는 curriculum sampling으로만 task를 생성하고
- task_utils.ipynb를 이용해서 curriculum mode의 task 파일을 random mode의 task 파일로 변환

In [5]:
domain = 'laptop'
mode = 'random'
num_task = 50000
n = 2
num_support = 80
num_query = 10

if domain == 'laptop':
    vocab = laptop_vocab_freq
    corpus = laptop
elif domain == 'rest':
    vocab = rest_vocab_freq
    corpus = rest

In [ ]:
%%time
tasks = []
for i, comb in enumerate(tqdm(vocab_sampler([tup[0] for tup in vocab], n, num_task, mode))):
    tasks.append(create_task(comb, corpus, num_support, num_query, mode))
    if (i+1)%1000 == 0:
        with open(f'task/task_{domain}_{mode}_{i+1}of{num_task}_{min_freq}_{n}_{num_support}_{num_query}.json', 'w') as f:
            json.dump(tasks, f)
        print(f'saved {i+1} tasks')

## Curriculum Sampling

#### Method
- Vocab의 조합을 100000개만큼 random sampling
- 각 조합의 cosine similarity를 기준으로 descending order로 sort
    - n이 3 이상일 때는 모든 <sub>n</sub>C<sub>2</sub>의 경우에 대한 similarity 값의 최댓값을 해당 조합의 similarity로 사용
- 상위 1000개의 조합(similarity가 높은)으로 task 생성

#### To-Do
- domain = 'laptop', n = 2, min_freq = 500, num_task = 1000 // 9시간 39분 소요
- domain = 'laptop', n = 3, min_freq = 500, num_task = 1000 // 13시간 27분 소요
- domain = 'rest', n = 2, min_freq = 500, num_task = 1000 // 11시간 38분 소요
- domain = 'rest', n = 3, min_freq = 500, num_task = 1000 // 16시간 49분 소요

#### 종강 이후
- domain = 'laptop', n = 2, min_freq = 500, num_task = 50000, num_support = 10, num_query = 10 // 5일 7시간 32분 소요
- domain = 'laptop', n = 3, min_freq = 500, num_task = 50000, num_support = 10, num_query = 10 // 8일 0시간 13분 소요
- domain = 'laptop', n = 4, min_freq = 500, num_task = 50000, num_support = 10, num_query = 10 // 10일 19시간 31분 소요
- domain = 'rest', n = 2, min_freq = 500, num_task = 50000, num_support = 10, num_query = 10 // 6일 11시간 13분 소요
- domain = 'rest', n = 3, min_freq = 500, num_task = 50000, num_support = 10, num_query = 10 // 9일 14시간 37분 소요
- domain = 'rest', n = 4, min_freq = 500, num_task = 50000, num_support = 10, num_query = 10 // 12일 20시간 47분 소요

In [4]:
domain = 'laptop'
mode = 'curriculum'
num_task = 50000
num_preview = 50000
n = 2
num_support = 10
num_query = 10

if domain == 'laptop':
    vocab = laptop_vocab_freq
    corpus = laptop
elif domain == 'rest':
    vocab = rest_vocab_freq
    corpus = rest

In [12]:
%%time
preview = vocab_sampler([tup[0] for tup in vocab], n, num_preview, mode)
preview
# default로 1000개만 표시됨

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 3min 29s, sys: 1.36 s, total: 3min 30s
Wall time: 3min 32s


[(['pixels', 'pixel'], 0.8635905),
 (['flawless', 'flaw'], 0.8582708),
 (['backlit', 'Backlit'], 0.8169732),
 (['charger', 'charge'], 0.78188175),
 (['preinstalled', 'uninstall'], 0.7625961),
 (['ACER', 'AC'], 0.7588044),
 (['machine', 'machines'], 0.71407336),
 (['photos', 'photoshop'], 0.6970816),
 (['taking', 'took'], 0.69201803),
 (['manage', 'managed'], 0.68650854),
 (['giving', 'gives'], 0.6839896),
 (['dvd', 'cd'], 0.6747162),
 (['newest', 'latest'], 0.67011064),
 (['push', 'pushing'], 0.65691096),
 (['concerns', 'concern'], 0.6531133),
 (['beware', 'malware'], 0.6422474),
 (['fill', 'filled'], 0.63609934),
 (['suggest', 'suggested'], 0.6202216),
 (['Using', 'using'], 0.615467),
 (['May', 'December'], 0.6145738),
 (['chipset', 'headset'], 0.60786545),
 (['feeling', 'feel'], 0.60761166),
 (['Thinkpad', 'keypad'], 0.60453516),
 (['fail', 'failed'], 0.60180455),
 (['need', 'needed'], 0.5832374),
 (['grandson', 'son'], 0.57868886),
 (['ive', 'i3'], 0.5668315),
 (['opening', 'closing

In [17]:
%%time
tasks = []
for i, comb in enumerate(tqdm(preview[:num_task])):
    tasks.append(create_task(comb, corpus, num_support, num_query, mode))
    if (i+1)%5000 == 0:
        with open(f'task/task_{domain}_{mode}_{num_task}_{min_freq}_{n}_{num_support}_{num_query}.json', 'w') as f:
            json.dump(tasks, f)
        print(f'saved {i+1} tasks')

  0%|          | 0/1000 [00:00<?, ?it/s]

saved 100 tasks
saved 200 tasks
saved 300 tasks
saved 400 tasks
saved 500 tasks
saved 600 tasks
saved 700 tasks
saved 800 tasks
saved 900 tasks
saved 1000 tasks
CPU times: user 9h 38min 21s, sys: 1min 27s, total: 9h 39min 49s
Wall time: 9h 39min 50s
